In [1]:
import numpy as 

# Реализую алгоритм Гаусса для решения систем линейных уравнений с помощью рекурсии

In [62]:
def gauss(A, n):
    #Проверим совместна ли матрица
    assert A.shape == (n,n+1), 'Матрица несовместна'
        
    #Проверим не нулевая ли матрица
    assert n != 0, 'Размерность не должна быть нулевой'
    
    #База рекурсии
    if n == 1:
        return np.array(A[0,1]/A[0,0])   
    
    #Копируем фрому матрицы
    A_new = np.zeros((n,(n+1)))
    
    #Запустим цикл по строкам, чтобы найти ту, у которой первый множитель не нулевой
    for row in range(n):
        if float(A[row,0]) != 0.:
            A_new[0] = A[row] #Такую строку перенесем наверх
            A_new[1:] = A[[i for i in range(n) if not i == row]] #И пересоберем матрицу
            break
        #Если весь столбец нулевой, вернем ошибку:
        assert float(A[row,0]) == 0. and row == n - 1, 'Одну из переменных невозможно определить'
            
    
    #Снова запустим цикл по строкам, чтобы привести множители первого столбца к нулю
    #Первую строку не включаем
    for row in range(1,n):
        #Если множитель не ноль, вычитаем из соответстующей ему строки преобразованную первую
        if A_new[row,0] != 0:
            A_new[row] -= A_new[0] * (A_new[row, 0]/A_new[0,0])
    
    #Запустим рекурсию над матрицей, не включающей первые строку и столбец, обработанные в этой итерации
    x_lower = gauss(A_new[1:, 1:], n - 1) 
    #Из базы рекурсии возвращаем массив x_lower с одним значением - первой вычисленной переменной
    
    #Промежуточные же итерации рекурсии принимают массив x_lower и считают переменную своего уровня:
    x = (A_new[0,-1] - np.sum(x_lower * A_new[0, 1:-1])) / A_new[0,0]
    
    #Добавляют ее в массив к прошлым (спереди) и возвращают дополненный массив наверх
    return np.hstack([x, x_lower])

In [73]:
n = 10
A = np.random.randint(10, size = [n,n + 1])

res = gauss(A, n)
check(A, res)


Дана матрица:
[[6 6 5 9 3 3 6 8 4 8 9]
 [4 9 3 8 9 5 0 7 6 0 2]
 [4 2 3 3 2 6 7 7 5 6 4]
 [7 3 9 3 5 4 8 0 1 6 5]
 [8 0 9 6 6 7 6 4 0 7 4]
 [1 2 7 4 7 2 6 2 8 5 1]
 [4 0 2 9 3 0 3 4 0 1 8]
 [2 7 4 6 8 2 9 8 0 0 4]
 [0 4 2 6 3 4 8 0 0 6 9]
 [3 2 7 2 6 0 7 6 8 3 5]] 
        
Подставим найденные значения переменных:
[  8.87749996  -2.23736151 -16.01905485  -5.32502654  14.67122439
  -7.77270616   0.39425839  -0.92997489  -1.56400566  10.97679988] 
        
Получаем: 
[[  53.26499973  -13.42416905  -80.09527424  -47.92523882   44.01367318
   -23.31811849    2.36555034   -7.43979908   -6.25602264   87.81439906
     9.        ]
 [  35.50999982  -20.13625357  -48.05716454  -42.60021228  132.04101954
   -38.86353081    0.           -6.5098242    -9.38403395    0.
     2.        ]
 [  35.50999982   -4.47472302  -48.05716454  -15.97507961   29.34244879
   -46.63623698    2.75980873   -6.5098242    -7.82002829   65.86079929
     4.        ]
 [  62.14249969   -6.71208452 -144.17149363  -15.97507

'Все правильно'

In [68]:
def check(A, res):
    if len(res) <=100:
        N = A[:,:-1] * res
        print(
            f"""
Дана матрица:
{A} 
        
Подставим найденные значения переменных:
{res} 
        
Получаем: 
{np.hstack([N,A[:,-1][:, np.newaxis]])} 
        
Просуммированная левая часть:
{np.sum(N, axis = 1)[:,np.newaxis]} 
        
Правая часть: 
{A[:,-1][:,np.newaxis]}
        
Разница:
{(A[:,-1] - np.sum(N, axis=1))[:,np.newaxis]}
        
Проверим, везде ли разница округляется до нуля с точностью до 10 знака: 
{np.all([not round(i, 10) for i in A[:,-1] - np.sum(N, axis=1)])}
"""
        )
        
        if np.all([not round(i, 10) for i in A[:,-1] - np.sum(N, axis=1)]):
            return 'Все правильно'
        return 'Ошибка'
    else:
        N = A[:,:-1] * res
        # Для бОльших значений размерности n погешность повышается, значит реализованный метод не оптимален
        if np.all([not round(i, 3) for i in A[:,-1] - np.sum(N, axis=1)]):
            return 'Все правильно'
        return 'Ошибка'